In [1]:
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sb

#from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
#import joblib
#from sklearn.model_selection import ShuffleSplit
#from sklearn.model_selection import KFold
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow import keras

from os import walk

import time
import datetime

%matplotlib inline

In [2]:
pathBase = 'I://Chuan//Documents//MyData//ProjNormalCAG//CoroBase//Arrangement//'
#pathBase = 'E://Chuan//Documents//Database//DICOM//CoroBase//Arrangement//'

theWhole = {}
# theWhole is dictionary containing views (keys) which the values are DICOM files

f = []
f_LCA_Caudal = []
mypath_LCA_Caudal = pathBase + 'LCA_Caudal'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Caudal):
    f.extend(filenames)
for l in f:
    f_LCA_Caudal.append(mypath_LCA_Caudal + '\\' + l) 
    
f = []
f_LCA_Cranial = []
mypath_LCA_Cranial = pathBase + 'LCA_Cranial'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Cranial):
    f.extend(filenames)
for l in f:
    f_LCA_Cranial.append(mypath_LCA_Cranial + '\\' + l) 
    
f = []
f_LCA_Cranial_LAO = []
mypath_LCA_Cranial_LAO = pathBase + 'LCA_Cranial_LAO'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Cranial_LAO):
    f.extend(filenames)
for l in f:
    f_LCA_Cranial_LAO.append(mypath_LCA_Cranial_LAO + '\\' + l) 
    
    
f = []
f_LCA_Cranial_RAO = []
mypath_LCA_Cranial_RAO = pathBase + 'LCA_Cranial_RAO'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Cranial_RAO):
    f.extend(filenames)
for l in f:
    f_LCA_Cranial_RAO.append(mypath_LCA_Cranial_RAO + '\\' + l) 
    
f = []
f_LCA_Spider = []
mypath_LCA_Spider = pathBase + 'LCA_Spider'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Spider):
    f.extend(filenames)
for l in f:
    f_LCA_Spider.append(mypath_LCA_Spider + '\\' + l) 
    
f = []
f_RCA_Cranial = []
mypath_RCA_Cranial = pathBase + 'RCA_Cranial'
for (dirpath, dirnames, filenames) in walk(mypath_RCA_Cranial):
    f.extend(filenames)
for l in f:
    f_RCA_Cranial.append(mypath_RCA_Cranial + '\\' + l) 
    
f = []
f_RCA_LAO = []
mypath_RCA_LAO = pathBase + 'RCA_LAO'
for (dirpath, dirnames, filenames) in walk(mypath_RCA_LAO):
    f.extend(filenames)
for l in f:
    f_RCA_LAO.append(mypath_RCA_LAO + '\\' + l) 
    

theWhole['LCA_Caudal'] = f_LCA_Caudal
theWhole['LCA_Cranial'] = f_LCA_Cranial
theWhole['LCA_Cranial_LAO'] = f_LCA_Cranial_LAO
theWhole['LCA_Cranial_RAO'] = f_LCA_Cranial_RAO
theWhole['LCA_Spider'] = f_LCA_Spider
theWhole['RCA_Cranial'] = f_RCA_Cranial
theWhole['RCA_LAO'] = f_RCA_LAO

In [3]:
viewConvert = {'LCA_Caudal': 0, 
               'LCA_Cranial': 1,
               'LCA_Cranial_LAO': 2,
               'LCA_Cranial_RAO': 3,
               'LCA_Spider': 4,
               'RCA_Cranial': 5,
               'RCA_LAO': 6}

In [4]:
theWhole['LCA_Cranial_LAO']

['I://Chuan//Documents//MyData//ProjNormalCAG//CoroBase//Arrangement//LCA_Cranial_LAO\\04A303A242F04B35BC889915B414E048.avi',
 'I://Chuan//Documents//MyData//ProjNormalCAG//CoroBase//Arrangement//LCA_Cranial_LAO\\04A303A242F04B35BC889915B414E048.dcm_average.npy',
 'I://Chuan//Documents//MyData//ProjNormalCAG//CoroBase//Arrangement//LCA_Cranial_LAO\\07FFCD52CE914F29B7E4BC877FDE039C.avi',
 'I://Chuan//Documents//MyData//ProjNormalCAG//CoroBase//Arrangement//LCA_Cranial_LAO\\07FFCD52CE914F29B7E4BC877FDE039C.dcm_average.npy',
 'I://Chuan//Documents//MyData//ProjNormalCAG//CoroBase//Arrangement//LCA_Cranial_LAO\\082257329B3A4223B258DCBAB5C11DEA.avi',
 'I://Chuan//Documents//MyData//ProjNormalCAG//CoroBase//Arrangement//LCA_Cranial_LAO\\082257329B3A4223B258DCBAB5C11DEA.dcm_average.npy',
 'I://Chuan//Documents//MyData//ProjNormalCAG//CoroBase//Arrangement//LCA_Cranial_LAO\\0AFD257E3761428881D7A2FFAC11F8D3.avi',
 'I://Chuan//Documents//MyData//ProjNormalCAG//CoroBase//Arrangement//LCA_Cranial_

In [5]:
start = time.time()

array = {}
i = 0

for view in theWhole.keys():
    array[view] = []
    for case in theWhole[view]:
        if case.split('.')[-1] == 'npy':
            arr = np.load(case)
            array[view].append(arr)

end = time.time()
print(end - start)

1.3075242042541504


### 2D to 1D

In [6]:
array_flat = {}

for view in array.keys():
    array_flat[view] = []
    for case in array[view]:
        case_flat = []
        for row in case:
            for column in row:
                case_flat.append(column)            
        array_flat[view].append(case_flat)

## Neural Network

In [7]:
x_sample_list = []
y_sample_list = []

for y, x in array_flat.items():    
    for x_item in x:
        x_sample_list.append(x_item)
        y_sample_list.append(y)

In [8]:
len(x_sample_list)

954

In [9]:
len(x_sample_list[0])

262144

In [10]:
512*512

262144

In [11]:
len(y_sample_list)

954

In [12]:
y_sample_list

['LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caudal',
 'LCA_Caud

In [13]:
y_sample_digital_list = []
for item in y_sample_list:
    y_sample_digital_list.append(viewConvert[item])

### Model

In [14]:
model = Sequential()
model.add(Dense(600, activation='relu', kernel_initializer='he_normal', input_shape=(262144, )))
model.add(Dense(300, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(200, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(100, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(80, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(30, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(7, activation='softmax'))

In [15]:
%load_ext tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [16]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training

In [17]:
start = time.time()

X_train = np.array(x_sample_list)
y_train = np.array(y_sample_digital_list)

model.fit(X_train, y_train, epochs=300, batch_size=70, verbose=1, callbacks=[tensorboard_callback])

end = time.time()

Epoch 1/400
 1/10 [==>...........................] - ETA: 0s - loss: 72.3166 - accuracy: 0.0600WARNING:tensorflow:From C:\Anaconda3\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
10/10 [==============================] - 16s 2s/step - loss: 25309.1914 - accuracy: 0.1153
Epoch 2/400
10/10 [==============================] - 15s 2s/step - loss: 22591.4219 - accuracy: 0.1551
Epoch 3/400
10/10 [==============================] - 15s 2s/step - loss: 16046.9453 - accuracy: 0.1426
Epoch 4/400
10/10 [==============================] - 15s 2s/step - loss: 15176.4160 - accuracy: 0.1855
Epoch 5/400
10/10 [==============================] - 15s 2s/step - loss: 13342.7080 - accuracy: 0.1184
Epoch 6/400
10/10 [==============================] - 15s 2s/step - loss: 7041.0259 - accuracy: 0.1614
Epoch 7/400
10/10 [===========

10/10 [==============================] - 15s 2s/step - loss: 27.2253 - accuracy: 0.7883
Epoch 79/400
10/10 [==============================] - 15s 1s/step - loss: 19.9832 - accuracy: 0.8470
Epoch 80/400
10/10 [==============================] - 15s 2s/step - loss: 18.7870 - accuracy: 0.8522
Epoch 81/400
10/10 [==============================] - 15s 2s/step - loss: 13.8230 - accuracy: 0.8763
Epoch 82/400
10/10 [==============================] - 15s 1s/step - loss: 34.7551 - accuracy: 0.7662
Epoch 83/400
10/10 [==============================] - 15s 1s/step - loss: 38.1548 - accuracy: 0.7484
Epoch 84/400
10/10 [==============================] - 15s 2s/step - loss: 24.5187 - accuracy: 0.8082
Epoch 85/400
10/10 [==============================] - 15s 2s/step - loss: 32.8629 - accuracy: 0.7704
Epoch 86/400
10/10 [==============================] - 15s 2s/step - loss: 29.0502 - accuracy: 0.8008
Epoch 87/400
10/10 [==============================] - 15s 1s/step - loss: 17.3288 - accuracy: 0.8564
Epo

10/10 [==============================] - 15s 1s/step - loss: 9.6771 - accuracy: 0.8658
Epoch 159/400
10/10 [==============================] - 15s 1s/step - loss: 13.2909 - accuracy: 0.8386
Epoch 160/400
10/10 [==============================] - 15s 2s/step - loss: 8.8595 - accuracy: 0.8868
Epoch 161/400
10/10 [==============================] - 15s 1s/step - loss: 5.0168 - accuracy: 0.9130
Epoch 162/400
10/10 [==============================] - 15s 1s/step - loss: 6.5558 - accuracy: 0.8795
Epoch 163/400
10/10 [==============================] - 15s 1s/step - loss: 6.3020 - accuracy: 0.8836
Epoch 164/400
10/10 [==============================] - 15s 1s/step - loss: 16.7917 - accuracy: 0.7966
Epoch 165/400
10/10 [==============================] - 15s 1s/step - loss: 13.8382 - accuracy: 0.8197
Epoch 166/400
10/10 [==============================] - 15s 1s/step - loss: 12.0155 - accuracy: 0.8470
Epoch 167/400
10/10 [==============================] - 15s 1s/step - loss: 12.9875 - accuracy: 0.8239

10/10 [==============================] - 15s 2s/step - loss: 1.9378 - accuracy: 0.1646
Epoch 239/400
10/10 [==============================] - 15s 1s/step - loss: 1.9375 - accuracy: 0.1646
Epoch 240/400
10/10 [==============================] - 15s 1s/step - loss: 1.9372 - accuracy: 0.1646
Epoch 241/400
10/10 [==============================] - 15s 2s/step - loss: 1.9370 - accuracy: 0.1646
Epoch 242/400
10/10 [==============================] - 15s 1s/step - loss: 1.9367 - accuracy: 0.1646
Epoch 243/400
10/10 [==============================] - 15s 1s/step - loss: 1.9364 - accuracy: 0.1646
Epoch 244/400
10/10 [==============================] - 15s 1s/step - loss: 1.9361 - accuracy: 0.1646
Epoch 245/400
10/10 [==============================] - 15s 1s/step - loss: 1.9359 - accuracy: 0.1646
Epoch 246/400
10/10 [==============================] - 15s 1s/step - loss: 1.9356 - accuracy: 0.1646
Epoch 247/400
10/10 [==============================] - 15s 1s/step - loss: 1.9354 - accuracy: 0.1646
Epoc

10/10 [==============================] - 15s 1s/step - loss: 1.9270 - accuracy: 0.1730
Epoch 320/400
10/10 [==============================] - 15s 1s/step - loss: 1.9270 - accuracy: 0.1730
Epoch 321/400
10/10 [==============================] - 15s 2s/step - loss: 1.9269 - accuracy: 0.1730
Epoch 322/400
10/10 [==============================] - 15s 2s/step - loss: 1.9269 - accuracy: 0.1730
Epoch 323/400
10/10 [==============================] - 15s 1s/step - loss: 1.9269 - accuracy: 0.1730
Epoch 324/400
10/10 [==============================] - 15s 1s/step - loss: 1.9268 - accuracy: 0.1730
Epoch 325/400
10/10 [==============================] - 15s 1s/step - loss: 1.9268 - accuracy: 0.1730
Epoch 326/400
10/10 [==============================] - 15s 1s/step - loss: 1.9268 - accuracy: 0.1730
Epoch 327/400
10/10 [==============================] - 15s 1s/step - loss: 1.9267 - accuracy: 0.1730
Epoch 328/400
10/10 [==============================] - 15s 2s/step - loss: 1.9267 - accuracy: 0.1730
Epoc

10/10 [==============================] - 15s 1s/step - loss: 1.9257 - accuracy: 0.1730


In [18]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 22004), started 4:18:19 ago. (Use '!kill 22004' to kill it.)

In [19]:
print(end - start)

4395.641496658325


### Prediction

In [20]:
pathBase = 'I://Chuan//Documents//MyData//ProjNormalCAG//CoroBase//Arrangement//test//'
#pathBase = 'I://Chuan//Documents//MyData//NormalCAG//CoroBase//Arrangement//test//'
#pathBase = 'E://Chuan//Documents//Database//DICOM//CoroBase//Arrangement//test//'

theWhole = {}

f = []
f_LCA_Caudal = []
mypath_LCA_Caudal = pathBase + 'LCA_Caudal'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Caudal):
    f.extend(filenames)
for l in f:
    f_LCA_Caudal.append(mypath_LCA_Caudal + '\\' + l) 
    
f = []
f_LCA_Cranial = []
mypath_LCA_Cranial = pathBase + 'LCA_Cranial'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Cranial):
    f.extend(filenames)
for l in f:
    f_LCA_Cranial.append(mypath_LCA_Cranial + '\\' + l) 
    
f = []
f_LCA_Cranial_LAO = []
mypath_LCA_Cranial_LAO = pathBase + 'LCA_Cranial_LAO'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Cranial_LAO):
    f.extend(filenames)
for l in f:
    f_LCA_Cranial_LAO.append(mypath_LCA_Cranial_LAO + '\\' + l) 
    
    
f = []
f_LCA_Cranial_RAO = []
mypath_LCA_Cranial_RAO = pathBase + 'LCA_Cranial_RAO'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Cranial_RAO):
    f.extend(filenames)
for l in f:
    f_LCA_Cranial_RAO.append(mypath_LCA_Cranial_RAO + '\\' + l) 
    
f = []
f_LCA_Spider = []
mypath_LCA_Spider = pathBase + 'LCA_Spider'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Spider):
    f.extend(filenames)
for l in f:
    f_LCA_Spider.append(mypath_LCA_Spider + '\\' + l) 
    
f = []
f_RCA_Cranial = []
mypath_RCA_Cranial = pathBase + 'RCA_Cranial'
for (dirpath, dirnames, filenames) in walk(mypath_RCA_Cranial):
    f.extend(filenames)
for l in f:
    f_RCA_Cranial.append(mypath_RCA_Cranial + '\\' + l) 
    
f = []
f_RCA_LAO = []
mypath_RCA_LAO = pathBase + 'RCA_LAO'
for (dirpath, dirnames, filenames) in walk(mypath_RCA_LAO):
    f.extend(filenames)
for l in f:
    f_RCA_LAO.append(mypath_RCA_LAO + '\\' + l) 
    

theWhole['LCA_Caudal'] = f_LCA_Caudal
theWhole['LCA_Cranial'] = f_LCA_Cranial
theWhole['LCA_Cranial_LAO'] = f_LCA_Cranial_LAO
theWhole['LCA_Cranial_RAO'] = f_LCA_Cranial_RAO
theWhole['LCA_Spider'] = f_LCA_Spider
theWhole['RCA_Cranial'] = f_RCA_Cranial
theWhole['RCA_LAO'] = f_RCA_LAO

In [21]:
start = time.time()

array = {}
i = 0

for view in theWhole.keys():
    array[view] = []
    for case in theWhole[view]:
        if case.split('.')[-1] == 'npy':
            arr = np.load(case)
            array[view].append(arr)

end = time.time()
print(end - start)

0.43982362747192383


### 2D to 1D

In [22]:
array_flat = {}

for view in array.keys():
    array_flat[view] = []
    for case in array[view]:
        case_flat = []
        for row in case:
            for column in row:
                case_flat.append(column)            
        array_flat[view].append(case_flat)

In [24]:
x_test_list = []
y_test_list = []

for y, x in array_flat.items():    
    for x_item in x:
        x_test_list.append(x_item)
        y_test_list.append(y)

In [25]:
y_test_digital_list = []
for item in y_test_list:
    y_test_digital_list.append(viewConvert[item])

In [26]:
X_test = np.array(x_test_list)
y_test = np.array(y_test_digital_list)

## Performance Measures

In [27]:
#y_predict = clf.predict(X_test)
#y_test = y_test

loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('Test Accuracy: %.3f' % acc)

Test Accuracy: 0.143


In [29]:
y_predict = model.predict(X_test)

In [30]:
y_predict

array([[0.17170212, 0.14427231, 0.08488871, 0.14529213, 0.15883617,
        0.13107069, 0.16393788],
       [0.17170212, 0.14427231, 0.08488871, 0.14529213, 0.15883617,
        0.13107069, 0.16393788],
       [0.17170212, 0.14427231, 0.08488871, 0.14529213, 0.15883617,
        0.13107069, 0.16393788],
       [0.17170212, 0.14427231, 0.08488871, 0.14529213, 0.15883617,
        0.13107069, 0.16393788],
       [0.17170212, 0.14427231, 0.08488871, 0.14529213, 0.15883617,
        0.13107069, 0.16393788],
       [0.17170212, 0.14427231, 0.08488871, 0.14529213, 0.15883617,
        0.13107069, 0.16393788],
       [0.17170212, 0.14427231, 0.08488871, 0.14529213, 0.15883617,
        0.13107069, 0.16393788],
       [0.17170212, 0.14427231, 0.08488871, 0.14529213, 0.15883617,
        0.13107069, 0.16393788],
       [0.17170212, 0.14427231, 0.08488871, 0.14529213, 0.15883617,
        0.13107069, 0.16393788],
       [0.17170212, 0.14427231, 0.08488871, 0.14529213, 0.15883617,
        0.13107069, 0.1

In [31]:
y_test

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6])

### Confusion Matrix (混淆矩阵)

In [35]:
cm = np.array(confusion_matrix(y_test, y_predict, labels=[0, 1, 2, 3, 4, 5, 6, 7]))
confusion = pd.DataFrame(cm, index=[0, 1, 2, 3, 4, 5, 6, 7],
                        columns=[0, 1, 2, 3, 4, 5, 6, 7])
confusion.to_csv('ConfusionMatrix_NN.csv')
confusion

ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets

In [ ]:
print(classification_report(y_test, y_predict))

In [ ]:
plt.figure(figsize=(8, 6))
heat_map = sb.heatmap(confusion, annot=True, 
                      cmap='coolwarm', 
                      xticklabels=confusion.columns, yticklabels=confusion.index, 
                      linewidths=.5, 
                      annot_kws={"size": 20})

In [ ]:
heat_map.figure.savefig("Heatmap_SVM.png", dpi=400)

## Save the Model

In [ ]:
c = joblib.dump(clf, 'trained_model.m')
print(c)

## Load the Model

In [ ]:
d = joblib.load('trained_model.m')
print(d)